### This script downloads and combines the 540 and 7a loan data and limits the dataset to SFDO counties.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
%matplotlib inline

### Load raw data

Save URLs for all the raw loan data.

In [2]:
d_504_url = 'https://c4sfdatascience.blob.core.windows.net/sba/FOIA%20-%20504%20(FY1991-Present).xlsx'
d_7a_91_99_url = 'https://c4sfdatascience.blob.core.windows.net/sba/FOIA%20-%207(a)%20(FY1991-FY1999).xlsx'
d_7a_00_09_url = 'https://c4sfdatascience.blob.core.windows.net/sba/FOIA%20-%207(a)%20(FY2000-FY2009).xlsx'
d_7a_10_present_url = 'https://c4sfdatascience.blob.core.windows.net/sba/FOIA%20-%207(a)%20(FY2010-Present).xlsx'

Load data into data frames (NOTE: this method of loading directly from the URL is pretty slow. You might consider downloading data onto local machine first.)

In [3]:
d_504 = pd.read_excel(d_504_url)

In [5]:
d_7a_91_99 = pd.read_excel(d_7a_91_99_url, skiprows=1) # 1st row has garbage

In [9]:
d_7a_00_09 = pd.read_excel(d_7a_00_09_url, skiprows=1) # 1st row has garbage

In [11]:
d_7a_10_present = pd.read_excel(d_7a_10_present_url)

Concatenate all the 7a files (the column names all match up)

In [13]:
d_7a = pd.concat((d_7a_91_99, d_7a_00_09, d_7a_10_present), axis = 0)

### Combine 7a and 504 data

Pull in 7a columns from the data dictionary:

In [15]:
cols_7a = list(pd.read_excel(os.path.join('Data', '7a_504_FOIA Data Dictionary.xlsx'), sheetname='7(a) Data Dictionary').iloc[:, 0])

We have two additional columns in the data file which are not present in the data dictionary:

In [16]:
print 'Number of columns in 7a dictionary:', len(cols_7a)
print 'Number of columns in 7a data:', len(d_7a_10_present.columns)

Number of columns in 7a dictionary: 32
Number of columns in 7a data: 34


In [17]:
for i in d_7a_10_present.columns:
    if i not in cols_7a:
        print i

SBADistrictOffice
CongressionalDistrict


Pull in 504 columns from the data dictionary:

In [18]:
cols_504 = list(pd.read_excel(os.path.join('Data', '7a_504_FOIA Data Dictionary.xlsx'), sheetname='504 Data Dictionary').iloc[:, 0])

We have two additional columns in the data file which are not present in the data dictionary (JobsSupported is just written differently):

In [19]:
print 'Number of columns in 540 dictionary:', len(cols_504)
print 'Number of columns in 540 data:', len(d_504.columns)

Number of columns in 540 dictionary: 34
Number of columns in 540 data: 36


In [20]:
for i in d_504.columns:
    if i not in cols_504:
        print i

SBADistrictOffice
CongressionalDistrict
JobsSupported


Check which columns that are in 504 are missing in 7a:

In [21]:
for i in d_504.columns: 
    if i not in d_7a_10_present.columns:
        print i

CDC_Name
CDC_Street
CDC_City
CDC_State
CDC_Zip
ThirdPartyLender_Name
ThirdPartyLender_City
ThirdPartyLender_State
ThirdPartyDollars


Check which columns that are in 7a are missing in 540:

In [22]:
for i in d_7a_10_present.columns:
    if i not in d_504.columns:
        print i

BankName
BankStreet
BankCity
BankState
BankZip
SBAGuaranteedApproval
RevolverStatus


Combine the 504 and 7a data sets.

In [30]:
combined = pd.concat((d_504, d_7a))

### Generate SFDO Dataset

Limit dataset to California only.

In [32]:
combinedca = combined[combined['BorrState'] == 'CA']

Limit dataset to the counties that belong to the SFDO (San Francisco District Office).

In [33]:
combinedca = combinedca[combinedca['ProjectCounty'].isin(['SANTA CRUZ', 'SANTA CLARA', 'SAN MATEO', 'ALAMEDA', 'CONTRA COSTA', 'MARIN',
                                           'SAN FRANCISCO', 'SOLANO', 'NAPA', 'SONOMA', 'LAKE', 'MENDOCINO', 'HUMBOLDT',
                                           'DEL NORTE'])]

Write the clean, relevant data set to CSV.

In [80]:
combinedca.to_csv(os.path.join('Data', 'Clean.csv'), encoding='utf-8', index = False)